## Read Data
train data: [training.json](../../../data/groundtruth_dataset/Binary_PIP_Classifier/training.json)

test data: [testing.json](../../../data/groundtruth_dataset/Binary_PIP_Classifier/testing.json)

In [ ]:
import json
import pandas as pd
def read_data(filepath):
    data = []
    with open(filepath,'r') as f:
        for line in f.readlines():
            data.append(json.loads(line))
    return data
#TODO: 
train_data_path = '../../data/groundtruth_dataset/Binary_Multiclass_PIP_Classifier/training.json'
test_data_path = '../../data/groundtruth_dataset/Binary_Multiclass_PIP_Classifier/testing.json'

train_data = read_data(train_data_path)
test_data = read_data(test_data_path)
train_df = pd.DataFrame(train_data)
eval_df = pd.DataFrame(test_data)

## Binary Text Classifier
### Training
model name: bert-base-multilingual-uncased

In [ ]:
#training
from simpletransformers.classification import ClassificationModel

EPOCH_NUM = 14
args={'num_train_epochs':EPOCH_NUM, 'output_dir': "outputs"}
text_model = ClassificationModel("bert", "bert-base-multilingual-uncased", args=args)

train_df_text = train_df
train_df_text['labels'] = train_df_text['labels'].replace(['0','1'],[0,1]) 

text_model.train_model(train_df_text, output_dir=f"output_text_train")

### Evaluation

In [ ]:
#evaluation
import os, re
from simpletransformers.classification import ClassificationModel

eval_df_text = eval_df
eval_df_text['labels'] = eval_df_text['labels'].replace(['0','1'],[0,1]) 
model_name = 'outputs/checkpoint-18452-epoch-14'
text_model = ClassificationModel("bert", model_name)
result_text, model_text_outputs, wrong_predictions = text_model.eval_model(eval_df_text)

wrong_texts = [each.text_a for each in wrong_predictions]
result_text['precision'] = result_text['tp'] / (result_text['tp'] + result_text['fp'])
result_text['recall'] = result_text['tp'] / (result_text['tp'] + result_text['fn'])
result_text['F1_score'] = 2 * result_text['precision'] * result_text['recall'] / (result_text['precision'] + result_text['recall'])
print(pd.DataFrame(result_text.items(), columns=['Key', 'Value']))